In [52]:
import os, sys


filedir = os.path.abspath(".")

currentdir = os.path.dirname(os.path.abspath(".")) # workaround to adding local modules

parentdir = os.path.dirname(os.path.abspath(""+currentdir))
print(f'CurrentDir: {currentdir}')
print(f'ParentDir: {parentdir}')

sys.path.insert(0, currentdir)

CurrentDir: /Users/renan.cristofalo/Documents/git/multimorbity_use
ParentDir: /Users/renan.cristofalo/Documents/git


In [53]:
!pip install openpyxl

# Importing Libraries

In [54]:
import pandas as pd
import numpy as np
import os.path as osp

from src.config import *

pd.options.display.max_columns = None
pd.options.display.float_format = "{:,.2f}".format

# Definitions

In [55]:
DATA_PATH = osp.join('..', 'data', 'raw','dataset_follow_up.xlsx')
OUTPUT_PATH = osp.join('..', 'data', 'processed','dataset_clean.csv')
TARGET = 'pronto_atendimento'


# Reading Data

In [56]:
df = pd.read_excel(DATA_PATH, sheet_name = 0, header = 0)

df.sample(10)


record_id         dt      hora             entrev  \
4753      13661 2021-11-28  16:10:00       Larissa rios   
2736       8912 2021-10-22  13:17:00   Danielle Campelo   
3941      11628 2021-11-11  15:12:00    Miriam Dallmann   
4459      12746 2021-11-22  14:14:00      samira gouvea   
2349       8029 2021-10-14  13:42:00              Elize   
3195      10071 2021-10-30  17:10:00  ana clara santana   
3052       9692 2021-10-28  16:58:00   Danielle Campelo   
3916      11582 2021-11-11  13:53:00        thais simon   
5058      14423 2021-12-01  17:19:00   matheus zambrano   
2018       6454 2021-10-05  16:01:00    Miriam dallmann   

                              setor domicilio endpdr  idaderc  sexorc  recusa  \
4753      431440705150091 - FRAGATA  Elegível    NaN      NaN     NaN     NaN   
2736  431440705130020 - TRÊS VENDAS  Elegível    NaN      NaN     NaN     NaN   
3941       431440705150017, FRAGATA  Elegível    NaN      NaN     NaN     NaN   
4459        431440705160053, CENTRO  Elegível    NaN      NaN     NaN     NaN   
2349        431440705180069 - AREAL  Elegível    NaN      NaN     NaN     NaN   
3195        431440705180072 - AREAL  Elegível    NaN      NaN     NaN     NaN   
3052  431440705130020 - TRÊS VENDAS  Elegível    NaN      NaN     NaN     NaN   
3916   431440705200042, SÃO GONÇALO  Elegível    NaN      NaN     NaN     NaN   
5058  431440705130030 - TRÊS VENDAS  Elegível    NaN      NaN     NaN     NaN   
2018       431440705160075 - CENTRO  Elegível    NaN      NaN     NaN     NaN   

      recusaout                   latitude                 longitude  \
4753        NaN -31,745,163,399,189,900.00 -5,237,110,111,863,290.00   
2736        NaN -31,706,761,485,416,200.00 -5,235,815,752,281,170.00   
3941        NaN              -3,175,427.00            -52,363,675.00   
4459        NaN            -317,606,077.00           -523,477,339.00   
2349        NaN            -317,461,344.00           -522,962,132.00   
3195        NaN -31,752,689,121,558,000.00 -5,230,387,214,676,110.00   
3052        NaN -31,708,622,312,636,900.00 -5,235,848,429,245,390.00   
3916        NaN             -31,749,522.00           -523,706,012.00   
5058        NaN            -317,124,954.00           -523,430,924.00   
2018        NaN              -3,175,052.00           -523,374,222.00   

                                  nmor   sexomor1  idademor1  \
4753                                 3   Feminino         32   
2736                                 4   Feminino         32   
3941  1 (apenas o próprio respondente)   Feminino         35   
4459                                 4  Masculino         39   
2349                                 4  Masculino         62   
3195                                 4   Feminino         59   
3052  1 (apenas o próprio respondente)  Masculino         78   
3916                                 2   Feminino         61   
5058                                 2   Feminino         59   
2018  1 (apenas o próprio respondente)   Feminino         86   

                   trabmor1 atforamor1   sexomor2  idademor2       trabmor2  \
4753               Trabalha        Não  Masculino       3.00            Não   
2736                    Não        NaN  Masculino      34.00       Trabalha   
3941               Trabalha        Sim        NaN        NaN            NaN   
4459               Trabalha        Não   Feminino      39.00       Trabalha   
2349                    Não        NaN   Feminino      60.00       Trabalha   
3195  Em auxilio do governo        NaN   Feminino      22.00            Não   
3052          Aposentado(a)        NaN        NaN        NaN            NaN   
3916          Aposentado(a)        NaN  Masculino      27.00  Aposentado(a)   
5058               Trabalha        Sim  Masculino      69.00  Aposentado(a)   
2018          Aposentado(a)        NaN        NaN        NaN            NaN   

     atforamor2   sexomor3  idademor3  trabmor3 atforamor3   sexomor4  \
4753        NaN  Mascul

# Procesing columns

In [57]:
# # !pip install unidecode
# import unidecode

# def standardize_column_name(col):
#     col = unidecode.unidecode(col)
#     return col.replace(' ', '_').lower()

# df.columns = [standardize_column_name(col) for col in df.columns]

# Data Cleaning

## Filtering Rows and Columns

In [85]:
print(f'df.shape pre-filter: {df.shape}')

# Filtering only matched rows (Baseline + Follow-up)
pdf = df.query("_merge == 'Matched (3)'")

# Get follow-up features to avoid data-leakage
follow_up_columns = df.columns[pdf.columns.get_loc('ACOMPANHAMENTO'):].to_list()

# Calculating target feature
pdf['pronto_atendimento'] = ((~pdf[pronto_atendimento].isin(['Não utilizou'])).sum(axis=1) > 0).astype(int)

# Drop unwanted columns

pdf = pdf.drop(drop_features + follow_up_columns, axis = 1)

print(f'pdf.shape post-filter: {pdf.shape}')

df.shape pre-filter: (5722, 875)
pdf.shape post-filter: (3502, 381)


/Users/renan.cristofalo/opt/miniconda3/envs/ml_saude/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


## Treating missing rows

In [86]:
# Adjusting null values
pdf[pdf['peso']>400] = np.nan
pdf[pdf['altura']>300] = np.nan
pdf['tpmfumo'].replace(['o','O', 'None', None], np.nan, inplace = True)


# replacing 999 with NaN
pdf.replace([999,'999','999.0', 'None', None,'none','99', '88','777','888','999','missing','NSA','nsa'], np.nan, inplace = True)

# Drop rows with all missing values
pdf.dropna(how = 'all', inplace = True)

# Feature Engineering

In [87]:
# Calculating multimorbidity
pdf['multi_calc'] = pdf[morbidades].isin(['sim',1,'Sim','1']).sum(axis=1)

# Adjusting Dtypes

In [88]:
# Apply dtypes
pdf = pdf.astype(dtypes_dict)

## Preprocessing

In [89]:
# dropping missing values on the target
pdf.dropna(subset=[TARGET], inplace=True)

In [90]:
# defining categorical features
# df[categorical_features] = df[categorical_features].astype(str)

# defining numeric features
# df[numeric_features] = df[numeric_features].astype('float')

# defining o target
pdf[TARGET] = pdf[TARGET].astype('int')

In [91]:
# Reevaluate data types
pdf = pdf.infer_objects()

# Final DataFrame

In [92]:
pdf.sample(10)

,setor,nmor,genero,cor,ler,escol,sit,altura,peso,conj,trab,rendapd,fumo,tafumo,tmfumo,tpafumo,tpmfumo,cigdia,audit1,audit2,audit3,audit4,audit5,audit6,audit7,audit8,audit9,audit10,abmicro,CW,ablav,abgel,abfree,ablavr,tv,tvcabo,abdvd,abmco,ac,abmoto,abauto,abseca,pecas,abban,abemd,abag,abpav,vz,md,abesc,renda,ajudarenda,ajudacovid___0,ajudacovid___1,ajudacovid___2,ajudacovid___3,ajudacovid___9,alirenda,cmdac,cmddn,dnali,dmali,mcom,cam,camt,aff,afft,afm,afmt,dcam,dcamt,dbici,dbicit,exer,exert,sent,deit,tvs,ppaf,atd,uatd,atdpt,jnprecisou,jnbuscou,jnconseguiu,jnondebuscou,jnmotivonao,plano,qplan,medenf,tamenf,tmmenf,imedenf,imedenfout,pref,postoqual,hmed,hmedout,nserv,tserv,tservout,eserv,eservout,introcovid,qcovid,tratcovid,rtestefam,tratcovidfam,covidfam,covidamg,vacina,dtvac,dtvac2,mvac,mvacout,vgp0,vgp1,cdt,ldt,mdt,omdt,autsa,comsa,idsa,sat,qsono,tsono,dcosta,bsec,asm,brq,enfs,dpoc,tpul___1,tpul___2,tpul___3,tpul___4,tpul___5,tpul___6,chiado,nmed,medind,medd,aqmed,inaf,ost,art,has,diab,pcor,park,rin,tire,glau,cat,alzh,inc,hepat,dor,out,outqual,dgp,qdgp___1,qdgp___2,qdgp___3,qdgp___4,qdgp___5,qdgp___6,qdgp___7,qdgp___8,qdgp___9,qdgp___10,qdgp___11,qdgp___12,qdgp___13,qdgp___14,qdgp___15,qdgp___16,qdgp___17,qdgp___18,qdgp___19,ang,avc,vavc,colt,epil,vepil,dep,ulc,itu,vitu,flu,vflu,pnm,vpnm,ca,taca,tmca,lca,ttca,pfarm,hemo,qtp,rtp,outt,qoutt,qued,vqued,acam,taacam,tmacam,inth,intht,poucoint9,pbaixo9,difsono9,cans9,semapetite9,sentiumal9,difconc9,lento9,ferir9,diftrab9,nerv7,imped7,preocup7,relax7,agit7,abor7,medo7,ajucama,ajumed,ajutare,ajuref,pessoamt,amor,abraco,queame,pessoamta,ouvir,confiar,compart,comprob,pessoaem,conselho,infajuda,quercons,sugprobm,pessoainf,fazagrad,discab,relax,divertir,pessoasp,ppeso,fraco,devg,menosaf,difcoisa,esfo,atvpd,banho,vestir,cama,lugares,banheiro,alimentar,cappd,mcj,dge,dsp,vlc,sozi,serv,idag,qserv,conj3,cor4,estudo4cat,cancer,fumo2,alc2,peso1,altura1,bmi,bmi4cat,obesidade,rendasetor,rensetor10,urg,rensetor5,_Irensetor1_2,_Irensetor1_3,_Irensetor1_4,_Irensetor1_5,_Irensetor1_6,_Irensetor1_7,_Irensetor1_8,_Irensetor1_9,_Irensetor1_10,banheiroabep,empabep,carroabep,microabep,lavaloucabep,gelaabep,freeabep,lavaroupaabep,dvdabep,microondasabep,motoabep,secaabep,chefeabep,aguaabep,pavabep,abep,abepcat,esc4j,sexj,cor2,abep4,j,ds,esperado,total,id4,mor2,idibge,pw,pw1,pop,estrato,idade,icovid,depre1,depre2,depre3,depre4,depre5,depre6,depre7,depre8,depre9,depress,depre_algo,depresscat,phq_sum_b,depre_cut,ansi1,ansi2,ansi3,ansi4,ansi5,ansi6,ansi7,ansiss,ansi_algo,gad_sum_b,ansi_cut,gad_sintomas,gad_severity,gad_severo,phq_severity,popw,is,id2,pros,multi,m2,m3,hospit,cmdac1,cmddn1,dnali1,dmali1,mcom1,inseg,testecovid,tcovid,asmbrq,pronto_atendimento,multi_calc
1437,431440705200030 - SÃO GONÇALO,1 (apenas o próprio respondente),Mulher cisgênera (1),Preta,sim,7º série do 1º grau,Trabalhando,157.00,78.00,casado(a) ou com companheiro(a),sim,Piorou muito,"não, nunca fumou",NaN,NaN,NaN,NaN,NaN,nunca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Sim, três",1.00,Não possui,"Sim, uma",Não possui,"Sim, uma","Sim, uma",Sim,Não possui,Não possui,Não possui,Não possui,Não possui,Não possui,3.00,1.00,nenhuma,Rede geral de distribuição,Terra/Cascalho,Segura,Não,Fundamental I completo / Fundamental II incomp...,Diminuiu muito,Não,Unchecked,Checked,Unchecked,Unchecked,Unchecked,Sim,Não,Não,Não,Não,Não,Nenhum,NaN,Nenhum,NaN,Nenhum,NaN,Nenhum,NaN,Nenhum,NaN,Não,NaN,180.00,20.00,20.00,Sim,"Sim, em um serviço de saúde",Posto de saúde,SUS,NaN,Sim,Sim,NaN,NaN,sim,Cartão de todos,não,NaN,NaN,NaN,NaN,sim,Ubs navegantes,"turno da manhã, tarde e noite, durante a semana",NaN,0.00,NaN,NaN,NaN,NaN,Não,NaN,NaN,Sim,Domicílio,Não,Não,Sim,Maio,Setembro,AstraZeneca/Oxford/FIOCRUZ,NaN,Não,Não,Mais de 2 anos,Serviço Público,Estava com dor,NaN,Ruim,Pior,Pior,8.00,Boa,7.00,Sim,Ocasionalmente,Não,Não,Não,Não,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,NaN,NaN,NaN,NaN,Não,Não,Sim,Sim,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,NaN

# Saving New DataFrame

In [93]:
pdf.to_csv(OUTPUT_PATH, index = False, header = True)